In [21]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

# local modules 
import sys
sys.path.insert(0, "../scripts")
from helpers import *

In [31]:
colorway = ['#702632',  '#495867', '#A4B494', '#912F40', "#81909E", "#F4442E", "#DB7C26", "#BB9BB0"]
pio.templates["cardinal"] = go.layout.Template(
    layout=go.Layout(
        colorway=colorway
    ))
# combine templates 
pio.templates.default = 'plotly_white+cardinal'

In [32]:
a00, a01 = import_desired_data("A", "15T")
b00, b01 = import_desired_data("B", "15T")
c00, c01 = import_desired_data("C", "15T")

In [33]:
def normalize_window_data(df):
    if len(df["Window Open"].unique()) > 1:
        return normalize_scale(df["Window Open"], df["Temp C"].min(), df["Temp C"].max())  
    else:
        return df["Window Open"]*df["Temp C"].max()

In [34]:
def add_var(df):
    if len(df["Window Open"].unique()) == 1:
        df["Window Open"][0] = 1 if df["Window Open"][0] == 0 else 0


    return df


In [54]:
def make_traces(fig, df, row, col):
    df = add_var(df)
    fig.add_trace(
        go.Scatter(
            x=df["DateTime"],
            y=df["Temp C"], 
            mode='lines',
            line=dict(width=4),
            showlegend=False
        ),
        row=row, col=col,
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(
            x=df["DateTime"],
            y=df["Window Open"], 
            # y=normalize_window_data(df),
            mode='lines',
            marker_color="gray",
            showlegend=False
        ),
        row=row, col=col,
        secondary_y=True,
    )

    fig.update_layout(
        yaxis2=dict(
            tickmode = 'array',
            tickvals = [0,1],
            ticktext = ['Closed', 'Open']
        ),
         yaxis4=dict(
            tickmode = 'array',
            tickvals = [0,1],
            ticktext = ['Closed', 'Open']
        ),
        xaxis_range=[df["DateTime"][8], df["DateTime"].iloc[-1]],
        xaxis2_range=[df["DateTime"][8], df["DateTime"].iloc[-1]],
    )

  



    return fig


In [60]:
def make_exp_plot(df0, df1):
    fig = make_subplots(rows=1, cols=2, 
            specs=[[{"secondary_y": True}, {"secondary_y": True}],], shared_yaxes=True)
    fig = make_traces(fig, df0, row=1, col=1)
    fig = make_traces(fig, df1, row=1, col=2)

#     fig.write_image(f"results/{exp_name}.png")

    return fig

In [61]:
make_exp_plot(a00, a01)


In [62]:
make_exp_plot(b00, b01)

In [63]:
fig = make_exp_plot(c00, c01)
fig.update_xaxes(
    dtick="D1",
    tickformat='%d %b')
